# Resuming stored ABC runs

In this examle, it is illustrated how stored ABC runs can be
loaded and continued later on.
This might make sense if you decide later on to run a couple more
populations for increased accuracy.

Let's start with the imports.

In [1]:
from pyabc import ABCSMC, Distribution, RV
import scipy as sp
from tempfile import gettempdir
import os

As usually, we start with the definition of the model,
the prior and the distance function.

In [2]:
def model(parameter):
    return {"data": parameter["mean"] + sp.randn()}

prior = Distribution(mean=RV("uniform", 0, 5))

def distance(x, y):
    return abs(x["data"] - y["data"])

db = "sqlite:///" + os.path.join(gettempdir(), "test.db")

We next make a new ABC-SMC run and also print the id of this run.
We'll use the id later on to resume the run.

In [3]:
abc = ABCSMC(model, prior, distance)
history = abc.new(db, {"data": 2.5})
run_id = history.id
print("Run ID:", run_id)

INFO:History:Start <ABCSMC(id=44, start_time=2019-11-10 23:36:40.114079, end_time=None)>


Run ID: 44


We then run up to 3 generations, or until the acceptance threshold 0.1
is reached -- whatever happens first.

In [4]:
history = abc.run(minimum_epsilon=.1, max_nr_populations=3)

INFO:ABC:Calibration sample before t=0.
INFO:Epsilon:initial epsilon is 1.2912959543955556
INFO:ABC:t: 0, eps: 1.2912959543955556.
INFO:ABC:Acceptance rate: 100 / 224 = 4.4643e-01.
INFO:ABC:t: 1, eps: 0.6651774908568672.
INFO:ABC:Acceptance rate: 100 / 326 = 3.0675e-01.
INFO:ABC:t: 2, eps: 0.33355947653050755.
INFO:ABC:Acceptance rate: 100 / 572 = 1.7483e-01.
INFO:History:Done <ABCSMC(id=44, start_time=2019-11-10 23:36:40.114079, end_time=2019-11-10 23:36:42.568825)>


Let's verify that we have 3 populations.

In [5]:
history.n_populations

3

We now create a completely new ABCSMC object.
We pass the same model, prior and distance from before.

In [6]:
abc_continued = ABCSMC(model, prior, distance)

To resume a run, we use the ``load`` method.
This loads the necessary data.
We pass to this method the id of the run we want to continue.

In [7]:
abc_continued.load(db, run_id)

In [8]:
abc_continued.run(minimum_epsilon=.1, max_nr_populations=1)

INFO:Epsilon:initial epsilon is 0.1529406275096243
INFO:ABC:t: 3, eps: 0.1529406275096243.
INFO:ABC:Acceptance rate: 100 / 1197 = 8.3542e-02.
INFO:History:Done <ABCSMC(id=44, start_time=2019-11-10 23:36:40.114079, end_time=2019-11-10 23:36:44.358045)>


Let's check the number of populations of the resumed run.
It should be 4, as we did 3 populations before and added another one.

In [9]:
abc_continued.history.n_populations

4

That's it. This was a basic tutorial on how to continue
stored ABC-SMC runs.